In [5]:
import numpy as np

num_states = 14
num_actions = 4
gamma = 1
theta = 0.5
prob = 0.1

a_desc = ["UP", "DOWN", "LEFT", "RIGHT"]
# 0 - UP
# 1 - DOWN
# 2 - LEFT
# 3 - RIGHT

maze = [['W','W','W','W','W','W'],
        ['W','S','S','S','S','W'],
        ['W','S','W','B','S','W'],
        ['W','S','W','S','B','W'],
        ['W','S','W','W','G','W'],
        ['W','B','S','S','W','W'],
        ['W','W','W','W','W','W']]

s_loc = np.zeros((num_states,2))
# print(s_loc)

# Initialize the state-value function V_0(s) = 0
v = np.zeros(num_states)

# Initialize the reward function
r = np.zeros((num_actions,num_states, num_states))

# Initialize the transition probability function
p = np.zeros((num_actions,num_states, num_states))

# Initialize the probability of taking each action in each state
# pi = np.ones((num_states, num_actions, num_states))

maze_col = len(maze[0])
maze_row = len(maze)


# Get the reward for the next state (s_prime)
def getReward(s_prime):
    reward = 0
    
    if(s_prime == 'B'):
        reward += -10
    elif(s_prime == 'G'):
        reward += 200
    
    reward += -1
    
    return  reward

# Map between the state and the location in the maze
def mapStateToMaze():
    s = 0
    for i in range(maze_row):
        for j in range(maze_col):
            if(maze[i][j] == 'S' or maze[i][j] == 'B'):
                s_loc[s] = [i,j]
                s += 1

mapStateToMaze()
s_loc = s_loc.astype(int)

print("The state location in the maze are: ")
print(s_loc)

# Get state from the location in the maze
def getStateFromMaze(i,j):
    k = np.where((s_loc == (i,j)).all(axis=1))
    if(k[0].size == 0):
        return 0
    else:
        return k[0][0]
            
# Get the next state given the current state and the action
def getNextState(s, a):
    s_prime = 0
    
    # The location of the current state in the maze
    i = s_loc[s,0]
    j = s_loc[s,1]
    
    if(a == 0):
        # Action is UP
        if(maze[i-1][j] != 'W'):
            s_prime = getStateFromMaze(i-1,j)
            p[a][s][s_prime] = 1 - prob
        else:
            s_prime = s
    elif(a == 1):
        # Action is DOWN
        if(maze[i+1][j] != 'W'):
            s_prime = getStateFromMaze(i+1,j)
            p[a][s][s_prime] = 1 - prob
        else:
            s_prime = s
    elif(a == 2):
        # Action is LEFT
        if(maze[i][j-1] != 'W'):
            s_prime = getStateFromMaze(i,j-1)
            p[a][s][s_prime] = 1 - prob
        else:
            s_prime = s
    elif(a == 3):
        # Action is RIGHT
        if(maze[i][j+1] != 'W'):
            s_prime = getStateFromMaze(i,j+1)
            p[a][s][s_prime] = 1 - prob
        else:
            s_prime = s
    
    return s_prime




The state location in the maze are: 
[[1 1]
 [1 2]
 [1 3]
 [1 4]
 [2 1]
 [2 3]
 [2 4]
 [3 1]
 [3 3]
 [3 4]
 [4 1]
 [5 1]
 [5 2]
 [5 3]]


In [7]:
for a in range(num_actions):
    for s in range(num_states):
        for s_prime in range(num_states):
            r[a][s][s_prime] = getReward(maze[s_loc[s_prime,0]][s_loc[s_prime,1]])
            # p[a][s][s_prime] = 
            


# Value Iteration

$
\begin{align}
V_{k+1} = \max_{a \in A} \sum_{S'} P(S'|S,a) \ \ \ [R(S,a,S')+V_k]
\end{align}
$

In [3]:
def getValueState(s, v, pi):
    
    # For each action
    for a in range(num_actions):
        



SyntaxError: incomplete input (1896551829.py, line 6)

## Policy Evaluation

$
\begin{align}
V^{\pi}(S) = \sum_{S'} P(S'|S,\pi(S)) \ \ \ [R(S,\pi(S),S')+V^{\pi}(S')]
\end{align}
$

In [ ]:
policy = np.zeros(num_states)

print(policy)

def policy_evaluation(env, policy, gamma=0.9, theta=0.01):
    """
    Evaluates a given policy in a dynamic programming setting.

    Args:
        env: The environment with a defined MDP (states, actions, rewards, transitions)
        policy: The policy to be evaluated (mapping of states to actions)
        gamma: The discount factor for future rewards.
        theta: The convergence threshold.

    Returns:
        np.array: An array of state-value estimates.
    """

    # num_states = env.nS  # Number of states
    value_func = np.zeros(num_states)  # Initialize state-value function

    while True:
        delta = 0  # Track the maximum change in value function
        for s in range(num_states):
            v = 0
            action = policy[s]
            for prob, next_state, reward, _ in env.P[s][action]:
                v += prob * (reward + gamma * value_func[next_state])
            delta = max(delta, abs(v - value_func[s]))
            value_func[s] = v  
        if delta < theta:
            break

    return value_func

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
